#Scrape Data from Indeed.ca

In [ ]:
import csv
from bs4 import BeautifulSoup

Get job posting summary website URL for each page

In [ ]:
def get_url(job_search, page):
  #For US https://www.indeed.com/jobs?q=Data+Analyst&l=
  #For Canada https://ca.indeed.com/jobs?q=" +job_search+"&start="+str(page*10)
  temp = "https://ca.indeed.com/jobs?q=" +job_search+"&start="+str(page*10)
  job_search = job_search.replace(' ','+')
  return temp.format(job_search)

In [ ]:
import requests
import numpy as np
import pandas as pd

In [ ]:
!pip install stop-words
#it use to avoid similar words and 

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=5806fbb22247ebb549e4dfdd767554fa13da40656d10e55afb2632b9b97e9d22
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer

en_stop = get_stop_words('en') #to lower capital
p_stemmer = PorterStemmer() #it use to avoid similar words and useless words

In [ ]:
#if find skill or experience in content, return to a list

def get_spec_skill_experience(a_list, dic):
  x = []
  for i in a_list:
    for j in dic:
      if j in i:
        x.append(i)

  return x

Read Description From Job Posting Page
If a sentence contain a "one" "two" "three" somthing like this, return to digital numbers. like"three years experience" to "3 years"

In [ ]:
def hasNumbers(inputString):
  return any(char.isdigit() for char in inputString) 

In [ ]:
#ref https://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers
#Word to number function for sentence

def text2int (textnum, numwords={}):
    if not numwords:
        units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]

        tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

        scales = ["hundred", "thousand", "million", "billion", "trillion"]

        #numwords["and"] = (1, 0)
        for idx, word in enumerate(units):  numwords[word] = (1, idx)
        for idx, word in enumerate(tens):       numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]

    textnum = textnum.replace('-', ' ')

    current = result = 0
    curstring = ""
    onnumber = False
    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if word not in numwords:
                if onnumber:
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
            else:
                scale, increment = numwords[word]

                current = current * scale + increment
                if scale > 100:
                    result += current
                    current = 0
                onnumber = True

    if onnumber:
        curstring += repr(result + current)

    return curstring

Now, we need to summarize the job posting content.
It will be two ways. firstly, find out the minimum experience they required generally. (ex. 2-5 years experience, take 2)
Degree required, (Bachelor's degree showing as Bachelor degree)
Also, find out which words appearing the most frequent

In [ ]:
import re
from gensim.summarization import keywords

from stop_words import get_stop_words
dic = ["math","sql","python","excel","experience","communication", "program","degree","qualification","skill"]

def readDescription(url):
  page = requests.get(url)
  src = page.content
  a = BeautifulSoup(src,'lxml')

  try:
    info = a.find('div', {"id":'jobDescriptionText'})
    info = [text for text in info.stripped_strings] #to list  .stripped_strings
    map(str.lower, info)
    #.text.replace('\n','')
  except:
    info =["none"]

 #remove useless word
  en_stop = get_stop_words('en')
  stopped_tokens = [i for i in info if not i in en_stop]

  info = [p_stemmer.stem(i) for i in stopped_tokens]
  fin_l = []
  fin_l = keywords(str(info), words = 8, split=True)

  info = get_spec_skill_experience(info, dic)
#GET NUMBER OF EXPERIENCE MINIMUM REQURIED

  number = 99 
  temp = []
  degree = "none"

  [temp.append(x) for x in info if x not in temp]

  for i in temp:
    i = text2int(i)
    #re.sub('[^A-Za-z0-9]+',' ', i)
    i = i.replace("'","") 
    i = i.replace("’","") 
    
    if ("experience" in i) & (hasNumbers(i))& ("year" in i):  #if a sentence contain experience, and years
      x = re.findall('(\d+)', i, re.IGNORECASE)
      if x:
        if int(min(x)) < number:
          number = int(min(x)) 

    if "degree" in i:  #if a sentence contain degree
      try:
        degree = re.findall("\w+ degree", i)
        degree =  ' '.join([str(elem) for elem in degree]) 
      except:
        if "education" in i:
          try:
            degree = re.findall("\w+ education", i)
            degree =  ' '.join([str(elem) for elem in degree]) 
          except:
            degree = "None"

  #Final Description Returned

  #fin_l = []
  #fin_l = keywords(str(info), words = 8, split=True)
  fin_l.insert(0, number) #INSERT MINIMUM experience data
  fin_l.insert(1, degree)

  return fin_l

#readDescription("https://ca.indeed.com/viewjob?jk=4b613f9bd434c66e&tk=1enq24fmot4im801&from=serp&vjs=3")


The website has different types to show the salary they offered
"$68,807 - $79,706 a year, weekly, day, hourly
This function is used to make all of them to an integer showing their hourly payment.

In [ ]:
def salary_num(stringW):
  Numberlist = re.findall(r"\d+\,?\d+\.?\d*", stringW)

  order = 0
  for originstring in Numberlist:
      if "," in originstring and "." not in originstring:  #for numbers like 68,870 and 27.33
          Numberlist.remove(originstring)
          newstring = originstring.replace(',' , '')
          Numberlist.insert(order,newstring)
          order +=1
      elif "." in originstring and "," in originstring:
          Numberlist.remove(originstring)
          newstring = originstring.replace(',' , '')
          Numberlist.insert(order,newstring)
          order +=1
#print(Numberlist)
  Total = 0
  for number in Numberlist:
      Total += float(number)
  temp = Total/len(Numberlist)
  if "week" in stringW:
    return round(temp/40,2)
  if "hour" in stringW:
    return round(temp,2)
  if "year" in stringW:
    return round(temp/2080,2)
  if "day" in stringW:
    return round(temp/8,2)
  else: return 99

#print(salary_num("$68,807 - $79,706 a year"))

In [ ]:
def readHtml(url): #read job posting url content for different pages
  page = requests.get(url)
  src = page.content
  a = BeautifulSoup(src,'lxml')

  div = a.find_all('div', {"class":'jobsearch-SerpJobCard'})
  #print(div)
  link = []
  title = []
  company = []
  salary =  []
  location = []
  description = []

  #print(summary)
  for i in div:
    title.append(i.find('a').text.strip())
    company.append(i.find('span', {"class":'company'}).text.strip())  #company name
    try:
      salary_test = i.find('span', {"class":'salaryText'}).text.strip() #salary if they have
      salary_test = salary_num(salary_test )
      salary.append(salary_test)
    except:
      salary.append("None")
    try:
      location.append(i.find('',{"class":'location accessible-contrast-color-location'}).text.strip()) #location if they have
    except:
      location.append("None")
    #print(title, company)
    details = i.find('a', {"target":'_blank'}) #read the url for inner detailed content
    if 'href' in details.attrs:
      x = details.attrs['href'].replace("/rc/clk", '')
      d_url = "https://ca.indeed.com/viewjob"+ x  #find href
      link.append(d_url )
      description.append(readDescription(d_url))  #!!!! summerize detailed description 
    else:
      link.append("None")
      description.append("None")


  job = {
      'title' : title,
      'company' : company,
      'salary': salary,
      'link':link,
      'description': description,
      'location':location
  }
  return job   #return a dic contain all infomation

#readHtml("https://ca.indeed.com/jobs?q=Data+Analyst&start=10")


#Export Data
Due to Operation limit, it will run serveral times, to export "1.csv","2.csv" etc.

In [ ]:
final_job_list = pd.DataFrame()
for i in range(0,20):   #in this case, page 0-19.
  url = get_url('Data Analyst', i)
  try:
    dp = pd.DataFrame(readHtml(url))
  except:
    dp = pd.DataFrame()
  #display(dp)
  final_job_list = final_job_list.append(dp, ignore_index = True) 



In [ ]:
final_job_list["Minimum Experience Required"] = final_job_list["description"].apply(lambda x: x[0])
final_job_list["Education Required"] = final_job_list["description"].apply(lambda x: x[1])
final_job_list["description"] = final_job_list["description"].apply(lambda x: x[2:])



In [ ]:
final_job_list.to_csv('0.csv')  #1,2,3,4